INDEXING

In [2]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = 'advanced-rag'
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ['GROQ_API_KEY'] = os.getenv("GROQQ_API_KEY")

MULTI - REPRESENTATION INDEXING

In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
loader = WebBaseLoader(
    "https://medium.com/@pankaj_pandey/introduction-to-retrieval-augmented-generation-rag-9209bf8a076d"
)
docs = loader.load()

loader = WebBaseLoader(
    "https://medium.com/humansdotai/an-introduction-to-ai-agents-e8c4afd2ee8f"
)
docs.extend(loader.load())

In [5]:
import uuid

from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("Summarize the following document:\n\n{doc}")
    | ChatGroq(model = "llama3-70b-8192")
    | StrOutputParser()
)

summaries = chain.batch(docs, {"max_concurrency": 1})

In [6]:
summaries

['The article introduces Retrieval-Augmented Generation (RAG), a framework that aims to improve the accuracy and reliability of Large Language Models (LLMs) by incorporating factual information during response generation. RAG addresses the drawbacks of LLMs, such as knowledge cutoff and response hallucination, by grounding them in external knowledge bases.\n\nThe purpose of RAG is to provide access to up-to-date facts and verifiable sources, increasing user trust in LLM-generated responses. RAG consists of two phases: retrieval and content generation. In the retrieval phase, algorithms search for and retrieve relevant information from external knowledge bases. In the generative phase, the LLM synthesizes an answer based on both the augmented prompt and its internal representation of training data.\n\nThe benefits of RAG include:\n\n* Accuracy and fact-checking: Ensuring LLM responses are based on reliable sources, allowing users to verify claims.\n* Reduced bias and hallucination: Limi

In [7]:
from langchain.storage import InMemoryByteStore
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

In [8]:
model_name = "BAAI/bge-small-en"
model_kwargs = {"device":"cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)

vectorstore = Chroma(
    collection_name= "summaries",
    embedding_function= hf_embeddings
)

store = InMemoryByteStore()
id_key = "doc_id"

retriever = MultiVectorRetriever(
    vectorstore= vectorstore,
    byte_store= store,
    id_key= id_key
)
doc_ids = [str(uuid.uuid4()) for _ in docs]

summary_docs = [
    Document(
        page_content= s,
        metadata = {id_key: doc_ids[i]}
    )
        for i,s in enumerate(summaries)
]

retriever.vectorstore.add_documents(summary_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

C:\Users\xoxo3\AppData\Local\Temp\ipykernel_7492\2439637394.py:4: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceBgeEmbeddings(


c:\Users\xoxo3\OneDrive\Desktop\PYTHON\5_hr_rag_course\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\xoxo3\AppData\Local\Temp\ipykernel_7492\2439637394.py:10: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [9]:
doc_ids

['66969814-fac7-46d6-abc3-5bd4cbf5c9aa',
 'de92ccd9-4ee6-4669-b8e5-0f0000395b5d']

In [10]:
retriever.docstore.mget(doc_ids)

[Document(metadata={'source': 'https://medium.com/@pankaj_pandey/introduction-to-retrieval-augmented-generation-rag-9209bf8a076d', 'title': 'Introduction to Retrieval-Augmented Generation (RAG) | by Pankaj | Medium', 'description': 'The world is advancing rapidly, introducing new technologies and stacks in AI and other areas every day. Large Language Models (LLMs) are a significant innovation in this space. However, LLMs have…', 'language': 'en'}, page_content='Introduction to Retrieval-Augmented Generation (RAG) | by Pankaj | MediumSitemapOpen in appSign upSign inMedium LogoWriteSign upSign inMastodonIntroduction to Retrieval-Augmented Generation (RAG)Pankaj6 min read·Dec 16, 2023--ListenShareRAG systems aim to address the drawbacks of Large Language Models by incorporating factual information during response generation, mitigating issues such as knowledge cutoff and response hallucination.Retrieval Augmented Generation (RAG)The world is advancing rapidly, introducing new technologies

In [11]:
query = "What is agent"
sub_docs = vectorstore.similarity_search(query,k=1)
sub_docs[0]

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


Document(metadata={'doc_id': 'de92ccd9-4ee6-4669-b8e5-0f0000395b5d'}, page_content='The article provides an introduction to Artificial Intelligence (AI) Agents, which are digital systems that can perform tasks, make decisions, and interact with their environment like humans. Powered by machine learning, natural language processing, and other cutting-edge technologies, AI Agents can learn from data, adapt to new information, and execute complex functions autonomously.\n\nThe article explains that AI Agents are designed to understand, analyze, and respond to human input, constantly evolving to enhance their capabilities. They exist in various forms, from chatbots providing customer service to sophisticated robots created for healthcare and manufacturing.\n\nThe article delves into the world of AI Agents, exploring their functionalities, the technology behind their intelligence, their applications across industries, and the ethical considerations that arise as these agents become increasi

In [12]:
retrieved_docs = retriever.get_relevant_documents(query,n_results = 1)
retrieved_docs[0].page_content[0:500]

C:\Users\xoxo3\AppData\Local\Temp\ipykernel_7492\899682954.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query,n_results = 1)


'An Introduction to AI Agents. Artificial Intelligence Agents are the… | by Humans.ai | humansdotai | MediumSitemapOpen in appSign upSign inMedium LogoWriteSign upSign inhumansdotai·Humans.ai utilizes blockchain technology to align artificial intelligence with trust, creating a human-centric approach. Our company views AI not as a tool but as a companion, envisioning a collaborative future with humanity.An Introduction to AI AgentsHumans.ai7 min read·Dec 27, 2023--2ListenShareArtificial Intellige'